### import selfmod

In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
from sqlalchemy import create_engine, text
import yaml
import pandas as pd

current_dir = Path(os.getcwd())
BASE_DIR = current_dir.parent 

if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

CONFIG_PATH = BASE_DIR / "configs" / "experiment_config.yaml"
DATA_PROCESSED = BASE_DIR / "data" / "processed" / "cleaned_data.csv"

print(f"✅ environment is ready,root set to:{BASE_DIR}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ environment is ready,root set to:l:\Foundations of Machine Learning Frameworks\LinearRegressionArchitecture_Workshop


In [5]:
try:
    with open(CONFIG_PATH, "r", encoding="utf-8") as file:
        config = yaml.safe_load(file)
except UnicodeDecodeError:

    with open(CONFIG_PATH, "r", encoding="cp950") as file:
        config = yaml.safe_load(file)

db_url = config.get("database", {}).get("db_url")
target_table = config.get("database", {}).get("table_name", "robot_telemetry")

if not db_url:
    raise ValueError("❌ Database URL not found in config!")


engine = create_engine(db_url, pool_pre_ping=True)


ddl = f"""
CREATE TABLE IF NOT EXISTS {target_table} (
  id BIGSERIAL PRIMARY KEY,
  elapsed_seconds DOUBLE PRECISION NOT NULL,
  axis1_smooth DOUBLE PRECISION NOT NULL
);
CREATE INDEX IF NOT EXISTS idx_{target_table}_id ON {target_table}(id);
"""

with engine.begin() as conn:
    for stmt in ddl.strip().split(";"):
        s = stmt.strip()
        if s: conn.execute(text(s))


if not DATA_PROCESSED.exists():
    raise FileNotFoundError(f"❌ 找不到資料檔: {DATA_PROCESSED}")

df = pd.read_csv(DATA_PROCESSED)
load_df = df[["Elapsed_Seconds", "Axis1_Smooth"]].rename(columns={
    "Elapsed_Seconds": "elapsed_seconds",
    "Axis1_Smooth": "axis1_smooth"
})


with engine.begin() as conn:
    conn.execute(text(f"TRUNCATE TABLE {target_table} RESTART IDENTITY;"))

load_df.to_sql(target_table, engine, if_exists="append", index=False, method="multi", chunksize=1000)

print(f"✅ Table {target_table} ready and {len(load_df)} rows loaded successfully.")

✅ Table robot_telemetry ready and 39672 rows loaded successfully.
